# Attributes Classification by Seeing Faces of people

---

## Introduction

In this notebook i will implement the attributes classifier using the `celeba_dataset`. We can understand about this task by the following points.
- `Task` : If we think about task this task will fall under the **Computer Vision** task with **Supervise Learning**.
- `Classifier` : For the classifier we can say that this is a **multilabel classification** problem so we will use the same type of classifier.

### 0. Installing Required Libraries

In [ ]:
!pip install gdown

### 1. Importing Libraries

In this section of notebook we are just importing libraries required to complete this task. As you can see in following code cell i divided cell into 3 parts
(seperated by blank line) <br>In *first part* i am importing `tensorflow specific libraries`, <br>In *second part* i am importing `helping libraries but specific to task` and <br>In *thirt part* i am importing `helping libraries (general purpose)`

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Dropout, GlobalAveragePooling2D, Activation, Dense
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import gdown
import glob

import os

### 2. Dataset Exploration and Big Picture

In this section of notebook i'll try to explore dataset as much as i can so that i can
grab the big picture.

In [ ]:
IMGS_ATTR_PATH = '/kaggle/input/celeba-dataset/list_attr_celeba.csv'
IMGS_PATH = '/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba'

FACE_WEIGHT_VGG16 = 'https://github.com/rcmalli/keras-vggface/releases/download/v2.0/rcmalli_vggface_tf_notop_vgg16.h5'

MODEL_20_EPOCHS = 'https://drive.google.com/uc?id=18N6-jvBPCODlgyyo0YiPiPTUxeyGBk0q'
MODEL_NAME = 'model_20_epoch.h5'

In [ ]:
gdown.download(MODEL_20_EPOCHS, MODEL_NAME)

In [ ]:
def get_attr_names(dataframe):
    '''
    This function returns attr names using in dataset.
    
    Params:
        dataframe (pd.Dataframe)
        
    Return:
        attr_names (list): A list object containing the name of attribute name of celeb faces.
    '''
    attr_names = []
    attr_names = list(dataframe.columns)
    # removing images_id column
    del attr_names[0]
    
    return attr_names

def attr_to_onehot(array):
    '''
    This function just make attr values in array to onehot by changing -1 to 0.
    '''
    result = np.copy(array)
    result[result == -1] = 0
    return result

def split_dataset(ds, percentage=0.2):
    '''
    This function can split the dataset in two parts with given percentage.
    
    Params: 
        ds (np.array): Numpy array representing dataset.
        percentage (float, optional): Between 0 and 1.
        
    Returns:
        main (np.array): This array contains lenght of 1-percentage of total dataset.
        other (np.array): This array contains length of percentage of total dataset.
    '''
    cut_size = int(percentage * len(ds))
    cut_indices = np.random.randint(0, len(ds), size=(cut_size, ))
    
    other = ds[cut_indices]
    main = np.delete(ds, cut_indices, axis=0)
    
    print('--------------------------------------')
    print("TOTAL:\t{} \nFIRST:\t{} \nSECOND:\t{}".format(len(ds), len(main), len(other)))
    print('--------------------------------------')
    
    return main, other

def plot_attr(attr_names):
    f = plt.figure(figsize=(20, 15))
    for i, attr_name in enumerate(attr_names):
        f.add_subplot(4, 10, i+1)
        col = attr_df[attr_name]
        g = sns.barplot(x=col.value_counts().index, y=col.value_counts(normalize=True))
        g.set(ylim=(0, 1))
        plt.yticks([])
    plt.show()

In [ ]:
attr_df = pd.read_csv(IMGS_ATTR_PATH)
attr_df.head()

In [ ]:
attr_names = get_attr_names(attr_df)
plot_attr(attr_names)

In [ ]:
attr2idx = {v:i for i, v in enumerate(attr_names)}

attr2idx

In [ ]:
attr_np = np.array(attr_df)
attr_np.shape

In [ ]:
celeb_dataset = attr_to_onehot(attr_np)
celeb_dataset[:5]

In [ ]:
train_np, test_np = split_dataset(celeb_dataset, 0.1)

### 3. Dataset Proprocessing

In this section i'll convert the dataset to trainable form with respect to model.

In [ ]:
def load_to_tfds(array):
    """
    This function just take np.array and extract imgs_path and attr_list
    seperately and then make seperate tensorflow dataset for both. Return 
    tensorflow dataset by zipping these two generated datasets.

    Args:
        array (np.array): Numpy array representing dataset

    Returns:
        ds (tf.data.Dataset): Tensorflow dataset containing zipped image and 
                              Respective attributes list.
    """ 
    img_path = array[:, 0]
    img_attrs = array[:, 1:]
    
    path_ds = tf.data.Dataset.from_tensor_slices(img_path)
    attrs_ds = tf.data.Dataset.from_tensor_slices(img_attrs.astype(np.int16))
    
    ds = tf.data.Dataset.zip((path_ds, attrs_ds))
    
    return ds

def load_and_process_image(name, attrs, augment=False):
    """
    This function take name(of image) and attrs(of image) and returns
    loaded image with attrs.

    Args:
        name (tf.string): String representing the name of image.
        attrs (tf.int): List representing the attributes of image.

    Returns:
        (image, attrs): Loaded image with attributes list.
    """
    full_path = IMGS_PATH + os.sep + name
    image = tf.io.read_file(full_path)
    image = tf.io.decode_jpeg(contents=image, channels=3)
    image = tf.cast(image, tf.float32)
    image = tf.divide(image, 255.)
    image = tf.image.resize(image, size=(112, 112))
    if (augment):
        image = tf.image.random_flip_left_right(image)
    return image, attrs

def preprocess_ds(ds, batch_size=128):
    """
    Batch and prefetch batched thats it.

    Args:
        ds (tf.data.Dataset): Tensorflow dataset
        batch_size (int, optional): Number of items to be in single batch. Defaults to 128.

    Returns:
        ds: Tensorflow dataset
    """
    ds = ds.shuffle(256)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

In [ ]:
train_ds = load_to_tfds(train_np)
test_ds = load_to_tfds(test_np)

train_ds = train_ds.map(lambda x, y: load_and_process_image(x, y, True))
test_ds = test_ds.map(load_and_process_image)

train_ds = preprocess_ds(train_ds, batch_size=64)
test_ds = preprocess_ds(test_ds, batch_size=64)

### 4. Dataset Visualization

In this part of task we generally Visualize the data with respective labels. As this dataset contains images so i will plot images with
attributes.

In [ ]:
def get_label_string(label):
    '''
    This function return string representation of label list
    
    Params:
        label (list) : A onehot representation of labels
        
    Return:
        joined by \n string from every element of list
    '''
    true_label = np.array(list(attr2idx.keys()))[label==1]
    return ', '.join(true_label)

def plot_images(images, labels, pred_labels=[]):
    '''
    This function plot the images one by one in 1x5 grid with string representation of label as title of
    respective image.
    
    Params:
        images (np.ndarray) : images to plot
        labesl (np.ndarray) : labels to plot as title
        pred_labels (nd.ndarray) : predicted labels to compare with true labels
    '''
    for i, image in enumerate(images):
        _ = plt.figure(figsize=(4, 4))
        print('-----------------------')
        print('True Attributes:', get_label_string(labels[i]))
        if len(pred_labels) != 0:
            print('Predicted Attributes:', get_label_string(pred_labels[i]))
        plt.imshow(image)
        plt.xticks([])
        plt.yticks([])
        plt.show()
        print('-----------------------')

In [ ]:
images, labels = next(iter(train_ds))
images, labels = images.numpy()[:5], labels.numpy()[:5]

plot_images(images, labels)

In [ ]:
# Deleting the unused vars
del train_np, test_np

### 5. Model Training with Validation

This section is all about training the model.

In [ ]:
def vgg16_model():
    '''
    This function is used to create the model. In this model i'll be using vgg16 pretrained model on vggface dataset.
    Notice i'm using GlobalAveragePooling instead of Dense layer.
    
    Returns:
        classifier (tf.keras.Model) : Custom model with resnet_50 pretrained.
    '''
    vgg_features = VGG16(include_top=False, input_shape=(112, 112, 3), pooling='avg', weights=None)
    face_weights = tf.keras.utils.get_file('vgg16_notop_weights', FACE_WEIGHT_VGG16)
    vgg_features.load_weights(face_weights)
    for layer in vgg_features.layers:
        layer.trainable = False
    classifier = Sequential([
        vgg_features,
        Dense(len(attr2idx)),
        Activation('sigmoid')
    ])
    return classifier

def resnet50_model():
    '''
    This function is used to create the model. In this model i'll be using resnet50 pretrained model on imagenet dataset.
    Notice i'm using GlobalAveragePooling instead of Dense layer.
    
    Returns:
        classifier (tf.keras.Model) : Custom model with resnet_50 pretrained.
    '''
    resnet_features = ResNet50(include_top=False, input_shape=(112, 112, 3), pooling='avg')
    for layer in resnet_features.layers:
        layer.trainable = False
    classifier = Sequential([
        resnet_features,
        Dense(len(attr2idx)),
        Activation('sigmoid')
    ])
    return classifier

def initializer_model(optimizer, loss, metrics):
    '''
    This function declare and compile the model with given params.
    
    Params:
        optimizer (string) : Optimizer for model
        loss (string) : Loss to minimize
        metrics (list) : Metrics to visualize while training
        
    Returns:
        model (tf.keras.Model) : Compiled model
    '''
    model = resnet50_model()
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return model

def load_or_initialize_model(path, loss, optimizer='adam', metrics=['accuracy']):
    '''
    This function load checkpoint weights to the model if there is present any otherwise just initialize model.
    
    Params:
        path (string) : Path to the model checkpoint
        loss (string) : Loss to minimize
        optimizer (string, optional) : Optimizer for the model
        metrics (list, optional) : Metrics to visualize while training
        
    Returns:
        model (tf.keras.Model) : Loaded model if checkpoint present otherwise new instance of model.
    '''
    model = initializer_model(optimizer, loss, metrics)
    print('Initialized Model...')
    checkpoint_dir = os.path.dirname(path)
    if len(glob.glob('./*.h5')) > 0:
        model = keras.models.load_model(glob.glob('./*.h5')[0])
        print("Loaded pretrained Model...")
    if not os.path.exists(checkpoint_dir):
        os.mkdir(checkpoint_dir)
        print('Created checkpoint directory...')
    elif len(os.listdir(checkpoint_dir)) > 0:
        model.load_weights(path)
        print("Loading Weights...")
        
    return model

def generate_class_weights(attr_names):
    '''
    This function used attr_names and attr_df to generate class_weight for losses to deal with
    unbalanced data as you can see in above histograms. This should e noticed that function returning
    value_counts[-1] due to we are more concerned for minorities class and mapping that to weights.
    
    Params:
        attr_name (list) : list of columns of attr_df to generate class_weight
        
    Returns:
        weights (dict) : Generated weight dict mapping label index to weights.
    '''
    weights = dict()
    for i, attr_name in enumerate(attr_names):
        value_counts = dict(attr_df[attr_name].value_counts(normalize=True))
        weights[i] = value_counts[-1]
    total_sum = sum(list(weights.values()))
    weights = {k: v/total_sum for k, v in weights.items()}
    return weights

In [ ]:
checkpoint_filepath = './checkpoint/cp.ckpt'

# Hyperparameters
optimizer = 'adam'
bc_loss = 'binary_crossentropy'
metrics = ['binary_accuracy']
EPOCHS = 20

# Loading or Initializing model
model = load_or_initialize_model(checkpoint_filepath, bc_loss, optimizer, metrics)

In [ ]:
model.summary()

In [ ]:
# callback definitions
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)
model_checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True, verbose=1)
class_weights = generate_class_weights(attr_names)

# training
history = model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS, callbacks=[early_stopping, model_checkpoint],
                   class_weight=class_weights)
model.save('./model_vggface_loss_{}.h5'.format(history.history['val_loss'][-1]))

**Notice** there is big difference between `loss` and `val_loss` due to `class_weights` which sums up to **1**.

### 6. Model Interference and Visualization

In this section i will plot the images with there true labels as well as predicted labels.

In [ ]:
# Tuning Parameter
thershold = 0.4
#--------
batch = next(iter(test_ds))
test_images, test_labels = batch

# Predicting Labels
pred_labels = model.predict(test_images)

# Changing Predicted labels to One-Hot
pred_labels[pred_labels >= thershold] = 1
pred_labels[pred_labels < thershold] = 0

# Processing for plotting
test_images = test_images.numpy()[:5]
test_labels = test_labels.numpy()[:5]
pred_labels = pred_labels[:5]

# Finally Plotting
plot_images(test_images, test_labels, pred_labels)

If results are not impressive then not that bad also.

# Thanks for Reading this Notebook

Please **UpVote** this notebook if you like this it means a lot to me.<br>
**PS**: We should be together too. :)